# Setup and load data:

In [ ]:
use_pca = True

In [ ]:
%load_ext autoreload
%autoreload 2

from functions.functions import *
from functions.plotting import *
from functions.autoencoder import *
from functions.mdn import *
from keras import backend as K
from keras.callbacks import TerminateOnNaN, ModelCheckpoint
from keras.models import model_from_json
from livelossplot.keras import PlotLossesCallback
import livelossplot
import tensorflow as tf
import numpy as np
import os
import glob
from itertools import combinations
import matplotlib
plt.rcParams['animation.ffmpeg_path'] = '/project/hep/demers/mnp3/miniconda3/envs/choreo/bin/ffmpeg' # for using html5 video in Jupyter notebook
print(matplotlib.animation.writers.list()) # check that ffmpeg is loaded. if it's not there, use .to_jshtml() instead of .to_html5_video().

In [ ]:
setup_gpus()

In [ ]:
data = load_data('rnn_data/mariel_*')

# Check out the real data:

In [ ]:
frame = np.random.randint(0,data.full.X.shape[1]-50)
print("Starting from frame {}...".format(frame))

# HTML(animate(data.full.X[:,frame:,:], frames=100))
HTML(animate(data.selected.X[:,frame:,:], frames=100, edges=data.selected.edges, colors='black'))

# LSTM + MDN

Plot the data with x,y centering:

In [ ]:
frame = np.random.randint(0,data.full.X.shape[1]-50)
X = data.selected.X # 15 joints
X = X.swapaxes(0, 1)
X[:,:,0] = X[:,:,0] - np.mean(X[:,:,0], axis=0) + 0.5*np.ones(15)
X[:,:,1] = X[:,:,1] - np.mean(X[:,:,1], axis=0) + 0.5*np.ones(15)
HTML(animate(data.selected.X[:,frame:,:], frames=100, edges=data.selected.edges, colors='black'))

In [ ]:
if use_pca: 
    X = data.selected.X # 15 joints
    # Average frame-by-frame in (x,y):
    X = X.swapaxes(0, 1)
    X[:,:,0] = X[:,:,0] - np.mean(X[:,:,0], axis=0) + 0.5*np.ones(15)
    X[:,:,1] = X[:,:,1] - np.mean(X[:,:,1], axis=0) + 0.5*np.ones(15)
    X = X.reshape(X.shape[0],X.shape[1]*X.shape[2])

    # PCA time:
    from sklearn.decomposition import PCA
    pca = PCA(.95)
    pca_reduced_data = pca.fit_transform(X)
    print('PCA reduction to a {}-dimensional latent space.'.format(pca_reduced_data.shape[1]))
    print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))
    print(pca_reduced_data.shape)
    n_time, n_dims, n_verts  = pca_reduced_data.shape[0], 1, pca_reduced_data.shape[1]
    n_mixes = 6
    look_back = 128

    lstm_mdn = LSTM_MDN(cells = [512,512,512,512], n_verts=n_verts, n_dims=n_dims, n_mixes=n_mixes, look_back=look_back)

    train_X = []
    train_Y = []
    for i in range(look_back, n_time, 1):
        train_X.append( pca_reduced_data[i-look_back:i,:] ) # look_back, verts * dims
        train_Y.append( pca_reduced_data[i,:] ) # verts * dims
    train_X = np.array(train_X) # n_samples, lookback, verts * dims
    train_Y = np.array(train_Y) # n_samples, verts * dims
    
    print(train_X.shape, train_Y.shape)
    lstm_mdn.model.summary()
    
else:
    X = data.selected.X # 15 joints
    n_verts, n_time, n_dims = X.shape
    n_mixes = 3
    look_back = 10

    lstm_mdn = LSTM_MDN(cells = [10,10,10,10], n_verts=n_verts, n_dims=n_dims, n_mixes=n_mixes, look_back=look_back)
    train_X, train_Y = lstm_mdn.prepare_inputs(X, look_back=look_back)
    print(train_X.shape)
    print(train_Y.shape)
    lstm_mdn.model.summary()

In [ ]:
checkpoint_filepath="weights/weights-lstm_pca_test.h5"
checkpoint = ModelCheckpoint(checkpoint_filepath, monitor='acc', verbose=1, save_best_only=True, mode='max')

# Train the model:
lstm_mdn.model.fit(train_X, train_Y, 
                   epochs=100, 
                   batch_size=128, 
                   shuffle=False, 
                   verbose=1,
                   callbacks=[checkpoint, TerminateOnNaN()])

In [ ]:
trained_model = lstm_mdn

### Load a trained model + weights

In [ ]:
trained_model = model_from_json(open('models/model-32chor-rnn.json').read(), {'MDN': MDN, 'LSTM_MDN': LSTM_MDN})
trained_model.load_weights('weights/weights-32chor-rnn.h5')
trained_model.summary()

n_mixes = 6
look_back = 128

if use_pca:
    means = np.load('pca/rnn_pca_32-means.npy')
    components = np.load('pca/rnn_pca_32-components.npy')

### See how well the model can predict the next frame in the input sequence:

In [ ]:
# visualize how well the model learned the input sequence
n_frames = 100 # n frames of time slices to generate
output_dims = train_X.shape[2]
frame = np.random.randint(0,data.full.X.shape[1]-50)
print("Seeding with frame {}".format(frame))
frames = []

test_X = train_X[frame:frame+n_frames] # data to pass into forward prop through the model
y_pred = trained_model.model.predict(test_X) # output with shape (n_frames, (output_dims+2) * n_mixes )

# partition out the mus, sigs, and mixture weights
for i in range(n_frames):
    y = y_pred[i].squeeze()
    mus = y[:n_mixes*output_dims]
    sigs = y[n_mixes*output_dims:n_mixes*output_dims + n_mixes]
    alphas = y[-n_mixes:]
    # find the most likely distribution - then disregard that number and use the first Gaussian :)
    alpha_idx = np.argmax(alphas)
    alpha_idx = 0
    # pull out the mus that correspond to the selected alpha index
    positions = mus[alpha_idx * output_dims:(alpha_idx+1) * output_dims]
    frames.append(positions)

if use_pca:
    # With PCA:
    frames = np.dot(frames, components) + means
    print(frames.shape)
    lstm_predictions = frames.swapaxes(0,1)
    lstm_predictions = np.dstack((lstm_predictions[::3,:],lstm_predictions[1::3,:],lstm_predictions[2::3,:]))
else:
    # No PCA:
    frames = np.array(frames)
    lstm_predictions = np.dstack((frames.T[::3,:],frames.T[1::3,:],frames.T[2::3,:]))
    
HTML(animate_ghost(data.selected.X[:,frame:,:], lstm_predictions[:,:,:], frames=n_frames, edges=data.selected.edges, colors='blue', ghost_shift = 0.3))

### Now generate new sequences!

In [ ]:
n_frames = 128 # n frames of time slices to generate
frames = []
seed = np.random.randint(0, len(train_X)-1)
x = np.expand_dims(train_X[seed], axis=0)
print(' * seeding with', seed)

for i in range(n_frames):
    y = trained_model.model.predict(x).squeeze()
    mus = y[:n_mixes*output_dims]
    sigs = y[n_mixes*output_dims:-n_mixes]
    alphas = softmax(y[-n_mixes:])

    # select the alpha channel to use
    alpha_idx = np.argmax(alphas)

    # grab the mus and sigs associated with the selected alpha_idx
    frame_mus = mus.ravel()[alpha_idx*output_dims : (alpha_idx+1)*output_dims]
    frame_sig = sigs[alpha_idx] / 100

    # now sample from each Gaussian
    positions = [np.random.normal(loc=m, scale=frame_sig) for m in frame_mus]
    positions = frame_mus

    # add these positions to the results
    frames.append(positions)

    # pull out a new training example - stack the new result on
    # all values after the first from the bottom-most value in the x's
    start = x[:,1:,:]
    end = np.expand_dims( np.expand_dims(positions, axis=0), axis=0 )
    x = np.concatenate((start, end), axis=1)
    
frames = np.array(frames)

if use_pca:
    # With PCA:
    frames = np.dot(frames, components) + means
#     print(frames.shape)
    lstm_predictions = frames.swapaxes(0,1)
    lstm_predictions = np.dstack((lstm_predictions[::3,:],lstm_predictions[1::3,:],lstm_predictions[2::3,:]))
else:
    # No PCA:
    lstm_predictions = np.dstack((frames.T[::3,:],frames.T[1::3,:],frames.T[2::3,:]))
    
prompt_plus_generated_seq = np.concatenate((data.selected.X[:,seed:seed+look_back,:],lstm_predictions), axis=1)
HTML(animate_ghost(data.selected.X[:,seed:seed+look_back+n_frames:,:], prompt_plus_generated_seq, frames=look_back+n_frames, edges=data.selected.edges, colors='blue'))

In [ ]:
HTML(animate(lstm_predictions, frames=n_frames, edges=data.selected.edges, colors='blue'))